In [1]:
# foto splitsen in 4 van 32x32
# 

from SimpleCV import *
from IPython.display import HTML
import matplotlib.pyplot as plt
import pandas as pd
import cv2
import glob

In [2]:
disp = Display(displaytype='notebook')
size = 32

imagelist = []
invertedImagelist =[]
horizontalSplit=[]

# get all files and add them to tuple with the postal codes
for filename in glob.glob("../dataset-images/*.png"): 
    originalImage = Image(filename)
    image = originalImage.binarize().dilate().erode()
    invertedImage = originalImage.binarize().dilate().erode().invert()
    invertedImage =  invertedImage.embiggen( (128,40), Color.WHITE, (3,3))
    answer = filename[18:22]
    imagelist.append((answer, image))
    invertedImagelist.append((answer, invertedImage))
  
    
numbers = []
numbersInverted = []

# seperate individual number inverted images with the number
for invertedImage in invertedImagelist:
    for i in range(4):
        sizeHeight = 40
        sizeWidth = 32
        digit = invertedImage[0][(0 if i+1 == 1 else i):i+1]
        croppedImage = invertedImage[1].crop(i*sizeWidth, 0, sizeWidth, sizeHeight*(i+1))
        numbersInverted.append((digit, croppedImage))

# seperate individual number images with the number
for image in imagelist:
    for i in range(4):
        digit = image[0][(0 if i+1 == 1 else i):i+1]
        numbers.append((digit, image[1].crop(i*size, 0, size, size*(i+1) )))

In [3]:
# divide number images into pieces of 4x4 and create matrix
def createMatrix(image):
    size = 8
    m = np.zeros((4,4))
# hoeveel pixels in stukje zwart
    pieces = []
    for y in range(4):
        for x in range(4):
            crop = image.crop(x*size, y*size, size, size)
            pieces.append(crop)
            m[x][y] = 1 if crop.meanColor()[0] > 80 else 0
    #print m.T
    return m.T

In [4]:
# check if centroid pixel is black. If black return 1 else 0 
def isCentroidPixelBlack(image, centroid):
    img = image.crop(centroid[0], centroid[1], 1, 1)
    color = img.meanColor()
    return 0 if color[0] > 200 else 1

In [58]:
def getBlob(image):
    image.show()
    blobs = image.findBlobs()
    image.draw(blobs, autocolor=True)
    return blobs[0]

def splitImage(image, centroidY, height):    
    image.show()
    
    croppedImageTop = image.crop(0,0,height,centroidY)
    blobTop = getBlob(croppedImageTop)
    centroidTop = blobTop.centroid()
    pixelBlackTop = isCentroidPixelBlack(croppedImageTop,centroidTop)
#     croppedImageTop.scale(4).save(disp)
    
    croppedImageBottom = image.crop(0,centroidY, height,(height-centroidY))   
    blobBottom = getBlob(croppedImageBottom)
    centroidBottom = blobBottom.centroid()
    pixelBlackBottom = isCentroidPixelBlack(croppedImageBottom,centroidBottom)
#     croppedImageBottom.scale(4).save(disp)

    return {'pixelBlackTop':pixelBlackTop, 'pixelBlackBottom':pixelBlackBottom}

#duplicate code of createMatrix
def createHeatMap(image):
    size = 8
    m = np.zeros((4,4))
# hoeveel pixels in stukje zwart
    pieces = []
    for y in range(4):
        for x in range(4):
            crop = image.crop(x*size, y*size, size, size)
            pieces.append(crop)
            m[x][y] = crop.meanColor()[0]
    plt.imshow(m.T, cmap='Blues', interpolation='nearest')
    plt.show()

def createDataList(blobOverview, invertedDigit, invertedBlobCount):
    index =0
    if len(blobOverview) > 0:
        for bo in blobOverview:
            index+=1
            if invertedDigit == bo[0]:
                if invertedBlobCount == bo[1]:
                    bo[2] += 1
                    break
            elif index == len(blobOverview):
                blobOverview.append([invertedDigit, invertedBlobCount, 1])
                break
    else:
        blobOverview.append([invertedDigit, invertedBlobCount, 1])
        
    

In [62]:
numbersDict = {
    "matrix_top_row": [],
    "matrix_second_row": [],
    "matrix_third_row": [],
    "matrix_bottom_row": [],
    "blob_area" : [],
    "blob_centroid_x" : [],
    "blob_centroid_y" : [],
    "blob_angle" : [],
    "blob_height": [],
    "blob_width": [],
    "inverted_blob_count":[],
    "centroid_is_black": [],
    "centroid_top_is_black":[],
    "centroid_bottom_is_black":[],
    "label" : []
}

blobOverview = []

for number, invertedNumber in zip(numbers, numbersInverted):
    image = number[1]
    invertedImage = invertedNumber[1]
    invertedDigit = invertedNumber[0]
    invertedBlobCount = len(invertedImage.findBlobs())
    blob = getBlob(image)
    matrix = createMatrix(image)
    centroid = blob.centroid()
    split = splitImage(image, centroid[1], blob.height())
    #createHeatMap(image)
    #check if digit already exist  
    createDataList(blobOverview, invertedDigit, invertedBlobCount)
    
    numbersDict["matrix_top_row"].append(sum(matrix[0]))
    numbersDict["matrix_second_row"].append(sum(matrix[1]))
    numbersDict["matrix_third_row"].append(sum(matrix[2]))
    numbersDict["matrix_bottom_row"].append(sum(matrix[3]))
    numbersDict["blob_area"].append(blob.area())
    numbersDict["blob_centroid_x"].append(centroid[0])
    numbersDict["blob_centroid_y"].append(centroid[1])
    numbersDict["blob_angle"].append(blob.angle())
    numbersDict["blob_height"].append(blob.height())
    numbersDict["blob_width"].append(blob.width())
    numbersDict["inverted_blob_count"].append(invertedBlobCount)
    numbersDict["centroid_is_black"].append(isCentroidPixelBlack(image, centroid))
    numbersDict["centroid_top_is_black"].append(split.get('pixelBlackTop'))
    numbersDict["centroid_bottom_is_black"].append(split.get('pixelBlackBottom'))
    numbersDict["label"].append(number[0])
print blobOverview 
print len(blobOverview)

df = pd.DataFrame(numbersDict)
df = df[["matrix_top_row", 
         "matrix_second_row", 
         "matrix_third_row", 
         "matrix_bottom_row", 
         "blob_area", 
         "blob_centroid_x", 
         "blob_centroid_y",
         "blob_angle",
         "blob_height",
         "blob_width",
         "inverted_blob_count",
         "centroid_is_black",
         "centroid_top_is_black",
         "centroid_bottom_is_black",
         "label"]]
df = df.sort_values("label")

[['7', 1, 196], ['6', 2, 135], ['2', 1, 193], ['0', 2, 173], ['6', 1, 60], ['3', 1, 194], ['1', 1, 198], ['8', 2, 62], ['9', 2, 144], ['8', 3, 91], ['7', 2, 3], ['5', 1, 184], ['0', 1, 4], ['8', 1, 27], ['9', 1, 60], ['4', 1, 171], ['3', 2, 5], ['5', 2, 3], ['4', 2, 14], ['2', 2, 2]]
20


In [ ]:
# TEST : 4686: too many blobs

testImage = Image("../dataset-images/4686.png")
testImage = testImage.binarize().dilate().erode()
blobs = testImage.findBlobs()
testImage.draw(blobs, autocolor=True)
testImage.save(disp)
print blobs.area()

testNums = []

for i in range(4):
    testNums.append(testImage.crop(i*size, 0, size, size*(i+1) ))

In [77]:
df.to_csv("../dataset-numpy/dataset_features.csv", sep=',', encoding='utf-8')

In [78]:
df.describe()

,matrix_top_row,matrix_second_row,matrix_third_row,matrix_bottom_row,blob_area,blob_centroid_x,blob_centroid_y,blob_angle,blob_height,blob_width,inverted_blob_count,centroid_is_black,centroid_top_is_black,centroid_bottom_is_black
count,1920.000000,1920.000000,1920.000000,1920.000000,1920.000000,1920.000000,1920.000000,1920.000000,1920.000000,1920.000000,1920.000000,1920.000000,1920.000000,1920.000000
mean,1.843750,1.613021,1.588021,1.844271,291.653385,15.859298,15.380416,50.857330,29.997917,20.457292,1.377083,0.295312,0.211458,0.348958
std,0.493377,0.558918,0.586954,0.604153,67.428881,1.193617,1.869070,65.489537,0.064533,2.035446,0.574330,0.456302,0.408449,0.476765
min,0.000000,0.000000,0.000000,0.000000,153.000000,11.835737,10.582800,-87.878899,28.000000,10.000000,1.000000,0.000000,0.000000,0.000000
25%,2.000000,1.000000,1.000000,1.000000,244.000000,15.077516,14.116460,65.718203,30.000000,20.000000,1.000000,0.000000,0.000000,0.000000
50%,2.000000,2.000000,2.000000,2.000000,275.500000,15.797984,15.295089,85.236358,30.000000,20.000000,1.000000,0.000000,0.000000,0.000000
75%,2.000000,2.000000,2.000000,2.000000,326.000000,16.571982,16.354908,90.000000,30.000000,22.000000,2.000000,1.000000,0.000000,1.000000
max,3.000000,3.000000,3.000000,3.000000,512.000000,20.410788,20.841542,90.000000,30.000000,30.000000,3.000000,1.000000,1.000000,1.000000


In [ ]:
# #list with tuples (digit, invertbloblength, counter) 
# ['0', 1, 4]
# ['0', 2, 173]
# ['1', 1, 198]
# ['2', 1, 193]
# ['2', 2, 2]
# ['3', 1, 194]
# ['3', 2, 5]
# ['4', 1, 171]
# ['4', 2, 14]
# ['5', 1, 184]
# ['5', 2, 3]
# ['6', 1, 60]
# ['6', 2, 135]
# ['7', 1, 196]
# ['7', 2, 3]
# ['8', 1, 27]
# ['8', 2, 62]
# ['8', 3, 91]
# ['9', 1, 60]
# ['9', 2, 144]